<a href="https://colab.research.google.com/github/mitreilab/autonomousfuture/blob/master/Copy_of_Copy_of_reilAutomation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import OES Data

In [0]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as pp
!wget "https://www.dropbox.com/s/t4444pduqc7rtgs/recentyears.csv?dl=0" -q -O "recent.csv" 
!wget "https://www.dropbox.com/s/6gglwj5jedmfvhi/oldyears.csv?dl=0" -q -O "old.csv" 
#Download raw csv file from Dropbox and rename to "rawdatanaics.csv"
new = pd.read_csv("recent.csv",low_memory=False)
old=pd.read_csv("old.csv",low_memory=False)


## Import ONet Data ##

In [0]:
!wget "https://www.dropbox.com/s/5cjlzvxqtoq5mm4/combined.sqlite?dl=0" -q -O  "onet.sqlite"  
#Download raw SQLite file from Dropbox and rename to "onet.sqlite"
import sqlite3
onetConnection = sqlite3.connect('onet.sqlite')
cur = onetConnection.cursor()
query = "SELECT * FROM occupation_data";
occdata = pd.read_sql_query(query, onetConnection)
len(occdata)
# Onet surveyed 1109 occupations.


1109

## OES Data cleaning

##Link old occ code with occ code after 2010


In [0]:
!wget "https://www.dropbox.com/s/30fryvdkh5qnk1z/crosswalktable.csv?dl=0" -q -O "crosswalk.csv" 
match=pd.read_csv("crosswalk.csv",low_memory=False)


In [0]:
old.columns

In [0]:
# Build a dictionary for old code to be transformed into new
oldlist=match['2000 SOC code'].tolist()
newlist=match['2010 SOC code'].tolist()
keys = oldlist
values = newlist
matchbook= dict(zip(keys, values))
matchbook[113041]

113111

In [0]:
tobecorrect=old['occ_code'].tolist()
len(tobecorrect)

7166

In [0]:
for i in range(len(tobecorrect)):
  oldcode=tobecorrect[i]
  try:
    temp=matchbook[oldcode]
    tobecorrect[i]=temp
  except:
    continue
correctcode= tobecorrect   
  


In [0]:

correctcode=pd.DataFrame(correctcode)


In [0]:
old['occ_code']=correctcode

In [0]:
old.reset_index(drop=True, inplace=True)
new.reset_index(drop=True, inplace=True)
result = pd.concat([old,new])

In [0]:
old.shape

(7166, 26)

In [0]:
new.shape

(15628, 26)

In [0]:
result.reset_index(drop=True, inplace=True)

In [0]:
len(newlist)

792

In [0]:
df=result[result['occ_code'].isin(newlist)]
df.shape
# Now we finishe building the OLS dabatase
# There are 792  detailed ocupations in US


(21264, 26)

In [0]:
# After we choose 13 industries (using 4_digit NAISC)
# We have 528 detailed occupations
targetlist=df['occ_code'].unique()
len(targetlist)

528

In [0]:
df.head()

,year,naics,naics_title,occ_code,occ_title,group,tot_emp_num,emp_prse,pct_total,pct_rpt,...,h_pct75,h_pct90,a_pct10,a_pct25,a_median_num,a_pct75,a_pct90,naics_codedescription,occ_code-title,detailed
0,2007,236100,Residential Building Construction,111011,Chief Executives,NaN,2700.0,6.9,0.28,1,...,#,#,74180,115920,NaN,#,#,236100-Residential Building Construction,11-1011-Chief Executives,1
1,2007,236100,Residential Building Construction,111021,General And Operations Managers,NaN,16680.0,3.4,1.72,7,...,#,#,46810,65310,98290.0,#,#,236100-Residential Building Construction,11-1021-General And Operations Managers,1
2,2007,236100,Residential Building Construction,112021,Marketing Managers,NaN,1090.0,6.5,0.11,~,...,58.13,#,47930,64750,84200.0,120910,#,236100-Residential Building Construction,11-2021-Marketing Managers,1
3,2007,236100,Residential Building Construction,112022,Sales Managers,NaN,1540.0,5.8,0.16,1,...,67.45,#,49600,67100,95020.0,140290,#,236100-Residential Building Construction,11-2022-Sales Managers,1
4,2007,236100,Residential Building Construction,112031,Public Relations And Fundraising Managers,NaN,50.0,22.1,0.01,~,...,59.19,#,37600,57360,95690.0,123110,#,236100-Residential Building Construction,11-2031-Public Relations And Fundraising Managers,1


### Target the occupations that are 1. In Real Estate industry 2. Not General Group 3. Onet has surveyed


In [0]:
# !wget "https://www.dropbox.com/s/329edv8yrn3ik25/Occupation%20Data.csv?dl=0"-q -O  "Occupation Data.csv" 
# occ_onet=pd.read_csv("Occupation Data.csv",low_memory=False)
# #Download 'Occupation Data' CSV that changes the 8-digit Onet-code to 6-digit, ommiting the last two digits.
# del_occ_endwith_00_and_in_Onet=oes.loc[oes['occ_code'].isin(occ_onet['O*NET-SOC Code'])]
# #Delete the occupation if the code is not in 'Occupation Data'
# del_occ_endwith_00_and_in_Onet.shape

In [0]:
# target_occ=set(del_occ_endwith_00_and_in_Onet['occ_code'])
# target_occ=list(target_occ)
# target=pd.DataFrame({'target_occ': target_occ})
# len(target)
# # we target 558 occupations.

### Drop the columns in the oes.csv that contains duplicated or redundant data.

In [0]:
# del_occ_endwith_00_and_in_Onet.columns
# #drop the colomuns that contain duplicate information
# del_duplicated_column=del_occ_endwith_00_and_in_Onet.drop(['title_code','occ_title_code', 'occ_title_mid', 'year.1', 'naics.1', 'naics_title.1', 'occ_code.1', 'occ_title.1', 'group', 'naics_codedescription', 'year-naics-occ_code', 'year-naics-occ_code.1','year.2', 'naics.2', 'naics_title.2', 'occ_code.2', 'title_code.1', 'dominant_lifecycle.1', 'occ_title.2', 'occ_title_code.1', 'occ_title_mid.1','group.1', 'naics_codedescription.1', 'keepsy'],axis=1)
# # the columns # decrease to 37 from 61
# del_duplicated_column.shape

## Call BLS data by 6-digit occupation code

In [0]:
# try to extract data by occ_code 
dx = del_duplicated_column[del_duplicated_column['occ_code'] == '11-1011']
dx[['year','naics_title', 'tot_emp_num']].head(3)

In [0]:
# assign the matrix of "year" and "naics_code" to a specific occ_code, call it 'labour_economy_data'
# first, create a function that generate the lb_econ_matrix for an occ_code
def lb_econ (occ_code,field,data):
    # this function yield a data frame for a given occ_code and a field
    # get all unique naics titles in the data
    feature_list = data['naics'].unique()
    # get all unique year in the data
    years = sorted(data['year'].unique())    
    mt_result = pd.DataFrame(0.00, index = years, columns = feature_list)   
    # filter out all rows and columns that have the given occ_code and fields
    df_occ = data[data['occ_code'] == occ_code][['year','naics_title', field]]
    for index, row in df_occ.iterrows():
        try:
            mt_result.loc[row['year']][row['naics_title']] = float(row[field])
        except:
            mt_result.loc[row['year']][row['naics_title']] = np.nan
    return mt_result.round(2)


In [0]:
# An example of how to use the function
data1=del_duplicated_column
# the lb_Econ_features contains all the fields' name that we can choose from, there're 17 all together.
lb_econ_features=['occ_code','tot_emp_num', 'emp_prse', 'pct_total', 'pct_rpt', 'h_mean', 'a_mean', 'mean_prse', 'h_pct10', 'h_pct25', 'h_median', 'h_pct75', 'h_pct90', 'a_pct10', 'a_pct25', 'a_median_num', 'a_pct75', 'a_pct90']
#try to input a field name called 'tot_emp_num'
aa=lb_econ (111011,'tot_emp_num',df)
aa.head()

In [0]:
df.columns

Index(['year', 'naics', 'naics_title', 'occ_code', 'occ_title', 'group',
       'tot_emp_num', 'emp_prse', 'pct_total', 'pct_rpt', 'h_mean', 'a_mean',
       'mean_prse', 'h_pct10', 'h_pct25', 'h_median', 'h_pct75', 'h_pct90',
       'a_pct10', 'a_pct25', 'a_median_num', 'a_pct75', 'a_pct90',
       'naics_codedescription', 'occ_code-title', 'detailed'],
      dtype='object')

# Object Oriented Programming (OOP)

In [0]:
# laborfeature include:'tot_emp_num', 'emp_prse', 'pct_total', 'pct_rpt', 'h_mean', 'a_mean',
#        'mean_prse', 'h_pct10', 'h_pct25', 'h_median', 'h_pct75', 'h_pct90',
#        'a_pct10', 'a_pct25', 'a_median_num', 'a_pct75', 'a_pct90',
#        'naics_codedescription'
# year include: 2007 to 2016
# industry include:
# 'Residential Building Construction',
# 'Nonresidential Building Construction',
# 'Land Subdivision',
# 'Foundation, Structure, and Building Exterior Contractors',
# 'Building Equipment Contractors',
# 'Building Finishing Contractors',
# 'Other Specialty Trade Contractors',
# 'Lessors of Real Estate',
# 'Offices of Real Estate Agents and Brokers',
# 'Activities Related to Real Estate',
# 'Architectural, Engineering, and Related Services',
# 'Specialized Design Services',
# 'Services to Buildings and Dwellings',
      
class occ(object):
  # add attributes to occ
  def __init__(self,code,title,year,industry,industrycode,name):
    self.code=code
    self.title=title
    self.year=year  
    self.industry=industry
    self.industrycode=industrycode
    self.name=name

#     yearemp,
#               emp_prse, pct_total, pct_rpt, h_mean, a_mean,
#        mean_prse, h_pct10, h_pct25, h_median, h_pct75, h_pct90,
#        a_pct10, a_pct25, a_median_num, a_pct75, a_pct90,
#       naics_codedescription,tasks,abilities,knowledge,skills,work_activities
#     self.yearemp=yearemp
#     self.emp_prse=emp_prse
#     self.pct_total=pct_total
#     self.pct_rpt=pct_rpt
#     self.h_mean=h_mean
#     self.a_mean=a.mean
#     self.mean_prse=mean_prse
#     self.h_pct10=h_pct10
#     self.h_pct25=h_pct25
#     self.h_median=h_median
#     self.h_pct75=h_pct75
#     self.h_pct90=h_pct90
#     self.a_pct10=a_pct10
#     self.a_pct25=a_pct25
#     self.a_median_num=a_median_num
#     self.a_pct75=a_pct75
#     self.a_pct90=a_pct90
#     self.naics_codedescription=naics_codedescription
#     self.laborfeature=laborfeature
#     self.task=tasks
#     self.abilities=abilities
#     self.knowledge=knowledge
#     self.skills=skills
#     self.work_activities=work_activities
  def __str__(self):
    return str(self.code)+str(self.title)+str(self.year)+str(self.industry+str(self.industrycode))
#   def__add__(self,other):
#     result=self.yearemp+other.yearemp
#     return result
#   def__eq__(self,other):
    
  # define methods:
  # total employment calculate employment number of each occ in all 13 industries
  # inputs for this function should be the 13 occ objects
  # with same code,title,year
  # with different industry,industry code,13 industries in the same year
#   def sumemp(self,other):
#     if self.code==other.code and self.year==other.year:
#       return self.yearemp+other.yearemp



## make all the objects, 528 times 10 years times 13 industries

In [0]:
row=df.iloc[0]
type(row[['year','naics']])


pandas.core.series.Series

In [0]:
a=occ(row['occ_code'],row['occ_title'],row['year'],row['naics_title'],row['naics'],'name1')
b=occ(row2['occ_code'],row2['occ_title'],row2['year'],row2['naics_title'],row2['naics'],'name2')

In [0]:
newlist.append(a.name)
newlist

In [0]:
# there are at most 68640 occ objects in total
# now we create them out and name them by 'occ+their occ_code'
listofocc=[]
all_occ=[]
for index, row in df.iterrows():
  #create a new object
  name=str(row['occ_code'])+str(row['occ_title'])+'in year'+str(row['year'])+'in industry'+str(row['naics'])
  temp=occ(row['occ_code'],row['occ_title'],row['year'],row['naics_title'],row['naics'],name)
  listofocc.append(temp.name)
  all_occ.append(temp)
  
  

In [0]:
# the list containing all the objects names is named as listofocc
# the list containing all the objects is named as all_occ
listofocc

In [0]:
len(listofocc)

21264

In [0]:
len(all_occ)

21264

In [0]:
!wget "https://www.dropbox.com/s/9ajd56nnhlrfo2m/Task%20Statements.csv?dl=0" -q -O "Task Statements.csv" 
Tasks = pd.read_csv("Task Statements.csv",low_memory=False)
!wget "https://www.dropbox.com/s/9ez75cfcojjpyur/Work%20Activities.csv?dl=0" -q -O "Work Activities.csv" 
Activities = pd.read_csv("Work Activities.csv",low_memory=False)

In [0]:
# Build a dictionary where
#   -  keys: the occ object
# Values : all the labor econ attributes and The O_net attributes


keys = all_occ

values =[]
# # We first define some functions that we need to search values from Onet
# # The function to search for tasks for an occpation:
# def gettask(occ_code,Tasks):
#   for index, row in Tasks.iterrows():
#     temp=Tasks[Tasks['occ_code']==occ_code][['Task ID','Task']]
#     onetvalue=temp.values.tolist()
#   return onetvalue
  
# # The function to search for tasks for an occpation:


In [0]:
# # Then we have a tasklist for each occupation, but this method is slow
# tasklist=gettask(111011,Tasks)

In [0]:
# tasklist[2]

[8825,
 'Analyze operations to evaluate performance of a company or its staff in meeting objectives or to determine areas of potential cost reduction, program improvement, or policy change.']

In [0]:
def gettask(occ_code,Tasks):
  for index, row in Tasks.iterrows():
    temp=Tasks[Tasks['occ_code']==occ_code][['Task ID','Task']]
  return temp

In [0]:
for index, row in df.iterrows():
  #create a sublist to be one element of the values list
  temp=row[['occ_code','tot_emp_num', 'emp_prse', 'pct_total', 'pct_rpt', 'h_mean', 'a_mean', 'mean_prse', 'h_pct10', 'h_pct25', 'h_median', 'h_pct75', 'h_pct90', 'a_pct10', 'a_pct25', 'a_median_num', 'a_pct75', 'a_pct90']]
  temptask=gettask(row['occ_code'])
  temp
  
  values.append(temp)

In [0]:
# The dictionary is named is manual
manual= dict(zip(keys, values))

In [0]:
manual[all_occ[1]]

occ_code        111021
tot_emp_num      16680
emp_prse           3.4
pct_total         1.72
pct_rpt              7
h_mean           55.15
a_mean          114710
mean_prse          1.9
h_pct10           22.5
h_pct25           31.4
h_median         47.26
h_pct75              #
h_pct90              #
a_pct10          46810
a_pct25          65310
a_median_num     98290
a_pct75              #
a_pct90              #
Name: 1, dtype: object

In [0]:
query = "SELECT * FROM occupation_data";
occdata = pd.read_sql_query(query, onetConnection)

In [0]:
targetlist=targetlist.tolist()

In [0]:
# built a occpationcode list to change 6digit to 8 digit and change the form
onetform=[]
for item in targetlist:
  string=str(item)
  temp=str(string[:1]) + '-' +str(string[2:])+'.00'
  onetform.append(temp)
onetform

<bound method NDFrame.head of        occ_code                              Title  Task ID  \
0        111011                   Chief Executives     8823   
1        111011                   Chief Executives     8831   
2        111011                   Chief Executives     8825   
3        111011                   Chief Executives     8826   
4        111011                   Chief Executives     8827   
5        111011                   Chief Executives     8824   
6        111011                   Chief Executives     8836   
7        111011                   Chief Executives     8832   
8        111011                   Chief Executives     8835   
9        111011                   Chief Executives     8833   
10       111011                   Chief Executives     8829   
11       111011                   Chief Executives     8837   
12       111011                   Chief Executives     8830   
13       111011                   Chief Executives     8839   
14       111011          